In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

# Step 1: Download historical stock data using Yahoo Finance
ticker = 'AAPL'  # Apple Inc. as an example
data = yf.download(ticker, start="2010-01-01", end="2023-01-01")

# Step 2: Preprocessing the data
# We will use 'Close' prices for prediction
data = data[['Close']]

# Normalize the stock price using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Step 3: Create the dataset with time window (for example, 60 previous days to predict the next day)
def create_dataset(data, time_step=60):
    X, y = [], []
    for i in range(len(data) - time_step - 1):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# Prepare the data with a time step of 60 days
time_step = 60
X, y = create_dataset(scaled_data, time_step)

# Reshaping X to be 3D for LSTM [samples, time steps, features]
X = X.reshape(X.shape[0], X.shape[1], 1)

# Step 4: Split the data into training and testing sets (80% train, 20% test)
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Step 5: Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=1))  # Predict the next closing price

model.compile(optimizer='adam', loss='mean_squared_error')

# Step 6: Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)

# Step 7: Predict on the test set
predicted_stock_price = model.predict(X_test)

# Invert the scaling
predicted_stock_price = scaler.inverse_transform(predicted_stock_price)
y_test = scaler.inverse_transform(y_test.reshape(-1, 1))

# Step 8: Visualize the results
plt.figure(figsize=(10,6))
plt.plot(y_test, color='blue', label='Real Stock Price')
plt.plot(predicted_stock_price, color='red', label='Predicted Stock Price')
plt.title(f'{ticker} Stock Price Prediction')
plt.xlabel('Time')
plt.ylabel('Stock Price')
plt.legend()
plt.show()

print("\nPredicted vs Real Stock Prices (Last 10 entries):")
for i in range(10):
    print(f"Day {i+1}: Predicted = {predicted_stock_price[i][0]:.2f}, Real = {y_test[i][0]:.2f}")
